In [1]:
import pandas as pd
import numpy as np
import re
%matplotlib inline 
import nltk
nltk.download('punkt')
import nltk  
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.metrics.distance  import edit_distance
nltk.download('words')
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
import string
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer
import nltk
nltk.download('stopwords')
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [61]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
# data = pd.read_csv('/content/drive/MyDrive/A1_dataset.csv')
data = pd.read_csv('C:/Users/HP/Downloads/A1_dataset.csv')
test_data = pd.read_csv('C:/Users/HP/Downloads/A2_test_dataset.csv')

In [4]:
import re
def text_cleaner(text):    
    # lower case text
    newString = text.lower()
    newString = re.sub(r"'s\b","",newString)
    
    # remove punctuations
    newString = re.sub(r'http\S+', '', newString)
    newString1 = re.sub("[^a-zA-Z]", " ", newString)    
    newString2 = re.sub(r'\d+', '', newString1)
    
    #return newString
    long_words=[]
    # remove short word
    stopword = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s?')
    text1 = stopword.sub(r'',str(newString2))

    for i in text1.split():
        if len(i)>=3:   
            long_words.append(i)
    return (" ".join(long_words)).strip()

In [6]:
# preprocess the text
data['new'] = data['TEXT'].apply(lambda x : text_cleaner(x))
#data_new = text_cleaner(data_text)

In [7]:
# preprocess the text
test_data['new'] = test_data['TEXT'].apply(lambda x : text_cleaner(x))
#data_new = text_cleaner(data_text)

In [8]:
test_data

,LABEL,DATE_TIME,TEXT,new
0,1,Fri May 29 22:24:26 2009,@mileycyrus cheer up miley whats wrong?,mileycyrus cheer miley whats wrong
1,1,Sun Jun 07 01:37:36 2009,Just got back in from The Belcourt. Saw &quot;...,got back belcourt saw quot fifth element quot ...
2,1,Wed May 13 23:41:18 2009,http://bit.ly/IQPPD with video,video
3,1,Sun May 31 16:43:58 2009,@chloebli heyy! how was your carnavilistic da...,chloebli heyy carnavilistic day woow made word
4,1,Fri May 29 10:36:59 2009,@deadlyseagal http://twitpic.com/66zex - Nice ...,deadlyseagal nice day
...,...,...,...,...
639,1,Sun Jun 07 12:52:53 2009,@jakesonaplane ha yup its gonna be a good day,jakesonaplane yup gonna good day
640,0,Sun May 03 20:00:34 2009,@DavidArchie Hey Love the episode but why only...,davidarchie hey love episode mins awesome loved
641,0,Mon Jun 01 05:14:30 2009,Sick--or allergytastic (not sure which!)--just...,sick allergytastic sure time miss saigon audit...
642,0,Sun Jun 07 08:16:06 2009,headache ... but sooooo worth it haha,headache sooooo worth haha


In [9]:
data

,LABEL,DATE_TIME,TEXT,new
0,0,Fri Jun 05 14:26:50 2009,About to get threaded and scared,get threaded scared
1,1,Thu May 14 10:13:55 2009,@awaisnaseer I like Shezan Mangooo too!!! I ha...,awaisnaseer like shezan mangooo one yesterday
2,1,Fri Jun 05 21:02:20 2009,worked on my car after work. showering then go...,worked car work showering going bed sooooooooo...
3,1,Sun Jun 14 22:25:52 2009,@Marama Actually we start this afternoon! I w...,marama actually start afternoon try something ...
4,1,Sun May 31 00:42:12 2009,@gfalcone601 Aww Gi.don't worry.we'll vote for...,gfalcone aww worry vote non stop coz love much
...,...,...,...,...
4282,1,Sat Jun 06 22:45:26 2009,@QandQ My performances on my CLEP tests. #qshock,qandq performances clep tests qshock
4283,0,Tue Jun 16 10:17:07 2009,"ugh no, rcn had all the true blood episodes on...",ugh rcn true blood episodes demand
4284,1,Fri May 01 22:00:42 2009,Just returned from the forest! Sarah (my merch...,returned forest sarah merchy lost keys woods day
4285,1,Sun Jun 07 02:09:46 2009,is proud of her dad and his piece of work. ( h...,proud dad piece work keep papa


In [10]:
a = data['new'].to_list()
np.savetxt(r'C:/Users/HP/Downloads/b.txt', a,fmt='%s')

In [ ]:
### Complete Bigram Language Model

import math
import random


class Ngram:
    def __init__(self, N):
        self.N = N
        self.bigram_prob = None

        # BIGRAM GOOD TURING RESULTS
        self.bigram_good_turing = None
        self.bigram_zero_occurence_prob = None
        self.bigram_good_turing_cstar = None


    def train(self, sentences):
        listOfBigrams = []
        bigramCounts = {}
        unigramCounts = {}
        listOfTrigrams = []
        trigramCounts = {}

        for sentence in sentences:
            words = sentence.split()
            for i in range(len(words)):
                if i < len(words) - 1:
                    listOfBigrams.append((words[i], words[i + 1]))
                    if (words[i], words[i + 1]) in bigramCounts:
                        bigramCounts[(words[i], words[i + 1])] += 1
                    else:
                        bigramCounts[(words[i], words[i + 1])] = 1

                    if i < len(words) - 2:
                        listOfTrigrams.append((words[i], words[i + 1], words[i + 2]))
                        if (words[i], words[i + 1], words[i + 2]) in trigramCounts:
                            trigramCounts[(words[i], words[i + 1], words[i + 2])] += 1
                        else:
                            trigramCounts[(words[i], words[i + 1], words[i + 2])] = 1

                if words[i] in unigramCounts:
                    unigramCounts[words[i]] += 1
                else:
                    unigramCounts[words[i]] = 1

        return listOfBigrams, unigramCounts, bigramCounts


    def get_bigram_prob(self, sentence):
            '''Return bigram probabilty of the sentence'''
            score = 1
            words = sentence.split()
            for i in range(len(words) - 1):
                unit = (words[i], words[i + 1])
                if unit in self.bigram_prob:
                    score *= self.bigram_prob[unit]
                else:
                    score *= 0
            return score

    def get_bigram_good_turing_prob(self, sentence):
        '''Return bigram probabilty of the sentence with good turing smoohting'''
        score = 1
        words = sentence.split()
        for i in range(len(words) - 1):
            unit = (words[i], words[i + 1])
            if unit in self.bigram_good_turing:
                score *= self.bigram_good_turing[unit]
            else:
                score *= self.bigram_zero_occurence_prob
        return score

    def prob(self, sentence):
        '''Returns the MLE probability of the given sentence. '''
        if self.N == 2:
            return self.get_bigram_prob(sentence)


    def sprob(self, sentence):

        '''Returns the smoothed probability of a given sentence. '''
        if self.N == 2:
            return self.get_bigram_good_turing_prob(sentence)


    def next(self, before_prior=None, prior=None):
        '''Samples a word from the conditional distribution of given context.'''
        if self.N == 2:
            return self.bigram_generate_next_word(prior)

    
    def calculate_bigram_prob(self, listOfBigrams, unigramCounts, bigramCounts):
        '''Assigns bigram probabilties'''
        bigram_prob = {}
        for bigram in listOfBigrams:
            bigram_prob[bigram] = (bigramCounts.get(bigram)) / (unigramCounts.get(bigram[0]))

        with open('./2gram.txt', 'w') as file:
            file.write('Bigram'  + 'Count' +  'Probability' + '\n')
            for bigrams in listOfBigrams:
                file.write(str(bigrams) + ' : ' + str(bigramCounts[bigrams]) + ' : ' + str(bigram_prob[bigrams]) + '\n')
        self.bigram_prob = bigram_prob
        return bigram_prob
    
    
    def calculate_bigram_prob_positive_negative(self, listOfBigrams, unigramCounts, bigramCounts, positive_negative, beta_component):
        '''Assigns bigram probabilties'''
        import math
        bigram_prob = {}
        
        for bigram in listOfBigrams:
            if positive_negative == 0:
                bigram_prob[bigram] = ((bigramCounts.get(bigram)) / (unigramCounts.get(bigram[0]))) / math.log(beta_component)
            else:
                bigram_prob[bigram] = ((bigramCounts.get(bigram)) / (unigramCounts.get(bigram[0]))) / (2 * math.log(beta_component))
        self.bigram_prob = bigram_prob
        return bigram_prob


    def good_turing_smooting(self, list_of_word_unit, word_unit_counts, total_number_of_word_unit, filename_debug, filename_result):
        '''Assigns good turing smoothed probabilties'''
        list_of_probabilities = {}
        bucket = {}
        c_star = {}
        p_star = {}
        list_of_counts = {}

        for word_unit in word_unit_counts.items():
            value = word_unit[1]  # Of time occurs
            if not value in bucket:
                bucket[value] = 1
            else:
                bucket[value] += 1

        bucket_list = sorted(bucket.items(), key=lambda t: t[0])
        zero_occurence_prob = bucket_list[0][1] / total_number_of_word_unit  # This many words occurs only once
        last_item = bucket_list[len(bucket_list) - 1][0]  # Most 266 occurs, occurs only one time

        # Set non existing # of words
        for x in range(1, last_item):
            if x not in bucket:
                bucket[x] = 0

        bucket_list = sorted(bucket.items(), key=lambda t: t[0])
        bucket_list_len = len(bucket_list)

        i = 1
        file = open(filename_debug, 'w')
        file.write("#NumberOfOccurences\t\t\tFrequency\n")
        for k, v in bucket_list:
            file.write(str(k) + " : " + str(v) + "\n")
            if i < bucket_list_len:
                if v == 0:
                    c_star[k] = 0
                    p_star[k] = 0

                else:
                    c_star[k] = (i + 1) * bucket_list[i][1] / v
                    p_star[k] = c_star[k] / total_number_of_word_unit

            else:
                c_star[k] = 0
                p_star[k] = 0

            i += 1

        file.close()
        for word_unit in list_of_word_unit:
            list_of_probabilities[word_unit] = p_star.get(word_unit_counts[word_unit])
            list_of_counts[word_unit] = c_star.get(word_unit_counts[word_unit])

        with open(filename_result, 'w') as file:
            file.write('Word Unit' +':'+ 'Count' + 'Probability' + '\n')

            for bigrams in list_of_word_unit:
                file.write(str(bigrams) + ' : ' + str(word_unit_counts[bigrams])
                           + ' : ' + str(list_of_probabilities[bigrams]) + '\n')

        return list_of_probabilities, zero_occurence_prob, list_of_counts

    def bigram_generate_next_word(self, prior, pos_neg=0):
        '''Genereates new word - bigram'''
        choose_list = []
        for elem in list(self.bigram_prob.keys()):
            if elem[0] == prior:
                choose_list.append(elem[1])
        while True:
            return random.choice(choose_list)

    def bigram_generate_sentence(self, maximum_iteration=7):
        '''Genereates a sentence using bigrams'''
        for i in range(7):
            sent = []
            word = "<s>"
            sent.append(word)
            for index in range(0, maximum_iteration):
                word = self.bigram_generate_next_word(word, 1)
                sent.append(word)
                if word == "</s>":
                    break
            if index == maximum_iteration - 1:
                sent.append("</s>")
            return sent

    def bigram_generate_sentences(self, number_of_sentences, maximum_iteration=7):
        '''Genereates some sentences using bigrams'''
        for i in range(7):
            sentences = []
            for i in range(0, number_of_sentences):
                sentence = self.bigram_generate_sentence(maximum_iteration)
                sentences.append(sentence)
            return sentences

    def ppl_bigram(self, sentences):
        '''Returns the bigram perplexity of the given list of sentences. Here smoothed values are used.'''
        counter = 0
        tmp = 0
        for i in range(len(sentences)):
            sentence = sentences[i]
            for j in range(len(sentence) - 1):
                unit = (sentence[j], sentence[j + 1])
                if unit in self.bigram_prob:
                    prob = self.bigram_prob[unit]
                else:
                    prob = 0
                if prob == 0:
                    tmp += 0
                else:
                    tmp += math.log(prob, 2)
                counter += 1
        # entropy = prob of each token / number of tokens
        try:
            entropy = -1 / counter * tmp
        except ZeroDivisionError:
            entropy = 0
        
        perplexity = math.pow(2, entropy)
        return perplexity


In [12]:
START = "<s>"
END = "</s>"

def read_data(filename):
    with open(filename, "r") as file:
        all_lines = file.readlines()

    return_lines = []
    for line in all_lines:
        return_lines.append(preprocess(line))

    return return_lines


def preprocess(line):
    line = line.replace("\n", "")
    line = START + " " + line + " " + END
    #line = " " +line + " "
    return line


def process_test_data(sentences):
    return_list = []
    for sentence in sentences:
        return_list.append(sentence.split())
    return 

In [ ]:
### Vader 

In [13]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter

# Will be using 2 LM models for finding the average of positive and negative accuracies

def vader_analysis(data: pd.DataFrame, positive_negative: int):
    sid = SentimentIntensityAnalyzer()
    data['scores'] = data['sentence'].apply(lambda review: sid.polarity_scores(review))
    data['compound']  = data['scores'].apply(lambda score_dict: score_dict['compound'])
    data['comp_score'] = data['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')
    polarity = data['comp_score'].tolist()
    c = Counter(polarity)
    if positive_negative == 0:
        return data, c['neg']
    else:
        return data, c['pos']
    
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [15]:
import numpy as np

train_filename = 'C:/Users/HP/Downloads/b.txt'
train_data = read_data(train_filename)
ngram = Ngram(2)
print("TRAINING STARTED...")
list_of_bigrams, unigram_counts, bigram_counts = ngram.train(train_data)
bigram_prob = ngram.calculate_bigram_prob(list_of_bigrams, unigram_counts, bigram_counts)
#one_gram_add_one_prob = ngram.onegram_add_one_smothing(unigram_counts)
ngram.bigram_good_turing, ngram.bigram_zero_occurence_prob, ngram.bigram_good_turing_cstar = ngram.good_turing_smooting(list_of_bigrams, bigram_counts, len(list_of_bigrams),
                                                                                                                            'C:/Users/HP/Downloads/good_turing_smooting_bigram.txt',
                                                                                                                            'C:/Users/HP/Downloads/good_turing_smooting_bigram_result.txt')
beta_components = list(range(2, 20))
print(beta_components)
max_accuracy = 0
best_positive = []
best_negative = []

for beta_component in beta_components:
    print("For beta_component = ", beta_component)
    # Working the negative part
    bigram_prob_pos_neg = ngram.calculate_bigram_prob_positive_negative(list_of_bigrams, unigram_counts, bigram_counts, 0, beta_component)
    bigram_sentences = ngram.bigram_generate_sentences(250)
    negative_sentences = []
    for sentence in bigram_sentences:
        joined_sentence = " ".join(sentence)
        negative_sentences.append(joined_sentence)
    negative_data = pd.DataFrame(data=negative_sentences, columns=['sentence'])
    negative_data, negative_count = vader_analysis(negative_data, 0)
    print("Count of correct negative sentences generated = ", negative_count)
    
    # Working the positive part
    bigram_prob_pos_neg = ngram.calculate_bigram_prob_positive_negative(list_of_bigrams, unigram_counts, bigram_counts, 1, beta_component)
    bigram_sentences = ngram.bigram_generate_sentences(250)
    positive_sentences = []
    for sentence in bigram_sentences:
        joined_sentence = " ".join(sentence)
        positive_sentences.append(joined_sentence)
    positive_data = pd.DataFrame(data=positive_sentences, columns=['sentence'])
    positive_data, positive_count = vader_analysis(positive_data, 1)
    print("Count of correct positive sentences generated = ", positive_count)
    
    positive_accuracy = (float)(positive_count/250)
    negative_accuracy = (float)(negative_count/250)
    accuracy = (float)((positive_accuracy+negative_accuracy)/2)
    if accuracy > max_accuracy:
        max_accuracy = accuracy
        best_positive = positive_data
        best_negative = negative_data
    print("Accuracy = ", accuracy)

TRAINING STARTED...
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
For beta_component =  2
Count of correct negative sentences generated =  54
Count of correct positive sentences generated =  196
Accuracy =  0.5
For beta_component =  3
Count of correct negative sentences generated =  62
Count of correct positive sentences generated =  199
Accuracy =  0.522
For beta_component =  4
Count of correct negative sentences generated =  53
Count of correct positive sentences generated =  206
Accuracy =  0.518
For beta_component =  5
Count of correct negative sentences generated =  52
Count of correct positive sentences generated =  199
Accuracy =  0.502
For beta_component =  6
Count of correct negative sentences generated =  63
Count of correct positive sentences generated =  199
Accuracy =  0.524
For beta_component =  7
Count of correct negative sentences generated =  48
Count of correct positive sentences generated =  198
Accuracy =  0.492
For beta_component =  8
Count of co

In [ ]:
### Save the model

In [16]:
import dill as pickle 
with open('a2_ngram_model.pkl', 'wb') as fout:
    pickle.dump(ngram, fout)

In [17]:
with open('a2_ngram_model.pkl', 'rb') as fin:
    ngram_loaded = pickle.load(fin)

In [18]:
p = best_negative['sentence']
print(ngram.ppl_bigram(p))

1.0


In [19]:
best_positive['sentence'] = best_positive['sentence'] .map(lambda x: x.lstrip('<s>').rstrip('</s>'))
best_negative['sentence'] = best_negative['sentence'] .map(lambda x: x.lstrip('<s>').rstrip('</s>'))

In [20]:
a = best_positive['sentence']
b = best_negative['sentence']

In [21]:
df = [a,b]

In [22]:
bigram_ppl_score = ngram.ppl_bigram(df)
print(bigram_ppl_score)

1.0


In [25]:
best_negative['LABEL'] = best_negative['comp_score'].map({'pos': 1, 'neg':0})

In [26]:
best_positive['LABEL'] = best_positive['comp_score'].map({'pos': 1, 'neg':0})

In [27]:
best_negative

,sentence,scores,compound,comp_score,LABEL
0,godlessgirl add none want shopping eating toast,"{'neg': 0.133, 'neu': 0.867, 'pos': 0.0, 'comp...",-0.0572,neg,0
1,madirigh damn sucks gotta,"{'neg': 0.565, 'neu': 0.435, 'pos': 0.0, 'comp...",-0.6369,neg,0
2,idk tell girl shut like thunder woke,"{'neg': 0.128, 'neu': 0.642, 'pos': 0.229, 'co...",0.2732,pos,1
3,terilynns amazing book school tommorow,"{'neg': 0.0, 'neu': 0.612, 'pos': 0.388, 'comp...",0.5859,pos,1
4,totzhatz sometimes late tired chill,"{'neg': 0.326, 'neu': 0.674, 'pos': 0.0, 'comp...",-0.4404,neg,0
...,...,...,...,...,...
245,lab work best massage,"{'neg': 0.0, 'neu': 0.543, 'pos': 0.457, 'comp...",0.6369,pos,1
246,chinese fooood mmmmm,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos,1
247,leeboardman cheers follow anyways,"{'neg': 0.0, 'neu': 0.617, 'pos': 0.383, 'comp...",0.4767,pos,1
248,diner guess wrong smile god little bit,"{'neg': 0.226, 'neu': 0.438, 'pos': 0.336, 'co...",0.1280,pos,1


In [28]:
best_negative = best_negative.drop(['scores'], axis=1)

In [29]:
best_negative = best_negative.drop(['compound'], axis=1)

In [30]:
best_negative = best_negative.drop(['comp_score'], axis=1)

In [31]:
best_negative

,sentence,LABEL
0,godlessgirl add none want shopping eating toast,0
1,madirigh damn sucks gotta,0
2,idk tell girl shut like thunder woke,1
3,terilynns amazing book school tommorow,1
4,totzhatz sometimes late tired chill,0
...,...,...
245,lab work best massage,1
246,chinese fooood mmmmm,1
247,leeboardman cheers follow anyways,1
248,diner guess wrong smile god little bit,1


In [32]:
best_positive = best_positive.drop(['scores'], axis=1)

In [33]:
best_positive = best_positive.drop(['compound'], axis=1)

In [34]:
best_positive = best_positive.drop(['comp_score'], axis=1)

In [35]:
best_positive

,sentence,LABEL
0,ahhhaaaa win good week rafting awesome amazing,1
1,elliematthewson complete legend sad okay well...,0
2,djmiles live support offline amp talked killed,0
3,overun phone ideas,1
4,shaunmichaelb summer needs better soon din ro...,1
...,...,...
245,jordanknight need songwriters,1
246,rainbowsleeve thanks mike well twit around real,1
247,eastcoastgamblr like drugs,1
248,wash whip beaver nuggets cooked terrific last,1


In [36]:
frames = [best_negative, best_positive]  
result = pd.concat(frames)
display(result)

,sentence,LABEL
0,godlessgirl add none want shopping eating toast,0
1,madirigh damn sucks gotta,0
2,idk tell girl shut like thunder woke,1
3,terilynns amazing book school tommorow,1
4,totzhatz sometimes late tired chill,0
...,...,...
245,jordanknight need songwriters,1
246,rainbowsleeve thanks mike well twit around real,1
247,eastcoastgamblr like drugs,1
248,wash whip beaver nuggets cooked terrific last,1


In [37]:
result['sentence'] = result['sentence'].apply(lambda s: text_cleaner(s))

In [38]:
result.to_csv("C:/Users/HP/Downloads/result.csv")

In [39]:
data = data.drop(['DATE_TIME'], axis=1)

In [40]:
data = data.drop(['TEXT'], axis=1)

In [41]:
data.rename(columns = {'new':'sentence'}, inplace = True)


In [42]:
data

,LABEL,sentence
0,0,get threaded scared
1,1,awaisnaseer like shezan mangooo one yesterday
2,1,worked car work showering going bed sooooooooo...
3,1,marama actually start afternoon try something ...
4,1,gfalcone aww worry vote non stop coz love much
...,...,...
4282,1,qandq performances clep tests qshock
4283,0,ugh rcn true blood episodes demand
4284,1,returned forest sarah merchy lost keys woods day
4285,1,proud dad piece work keep papa


In [43]:
frames1 = [result,data]  
result1 = pd.concat(frames1)
display(result1)

,sentence,LABEL
0,godlessgirl add none want shopping eating toast,0
1,madirigh damn sucks gotta,0
2,idk tell girl shut like thunder woke,1
3,terilynns amazing book school tommorow,1
4,totzhatz sometimes late tired chill,0
...,...,...
4282,qandq performances clep tests qshock,1
4283,ugh rcn true blood episodes demand,0
4284,returned forest sarah merchy lost keys woods day,1
4285,proud dad piece work keep papa,1


In [44]:
result1.to_csv("datasetb.csv")

In [45]:
a = pd.read_csv("datasetb.csv")

In [46]:
a

,Unnamed: 0,sentence,LABEL
0,0,godlessgirl add none want shopping eating toast,0
1,1,madirigh damn sucks gotta,0
2,2,idk tell girl shut like thunder woke,1
3,3,terilynns amazing book school tommorow,1
4,4,totzhatz sometimes late tired chill,0
...,...,...,...
4782,4282,qandq performances clep tests qshock,1
4783,4283,ugh rcn true blood episodes demand,0
4784,4284,returned forest sarah merchy lost keys woods day,1
4785,4285,proud dad piece work keep papa,1


In [ ]:
### dataset B contains A+500 (result1 here)

# **Part Evaluation**


In [47]:
# On Dataset B using CountVectorizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
MNB = MultinomialNB()

In [48]:
# On Dataset B using TfidfVectorizer
tfidf = TfidfVectorizer()
text_count_2 = tfidf.fit_transform(result1['sentence'])

#splitting the data in test and training
#from sklearn.model_selection() import train_test_split()
x_train, x_test, y_train, y_test = train_test_split(text_count_2, result1['LABEL'],test_size=0.25,random_state=5)

#defining the model
#compilimg the model -> we are going to use already used models GNB, MNB, CNB, BNB
#fitting the model
MNB.fit(x_train, y_train)
accuracy_score_mnb = metrics.accuracy_score(MNB.predict(x_test), y_test)
print('accuracy_score_mnb = '+str('{:4.2f}'.format(accuracy_score_mnb*100))+'%')

accuracy_score_mnb = 69.59%


In [49]:
# On Test data with training on Dataset B using TfidfVectorizer
test_text_count_2 = tfidf.transform(test_data['new'])
MNB = MultinomialNB()

# Fitting on all of the training data present
MNB.fit(text_count_2, result1['LABEL'])
predicted = MNB.predict(test_text_count_2)
accuracy_score = metrics.accuracy_score(predicted, test_data['LABEL'])
print(str('{:04.2f}'.format(accuracy_score*100))+'%')

90.22%


In [50]:
# On Dataset A using TfidfVectorizer
tfidf = TfidfVectorizer()
text_count_2 = tfidf.fit_transform(data['sentence'])

#splitting the data in test and training
#from sklearn.model_selection() import train_test_split()
x_train, x_test, y_train, y_test = train_test_split(text_count_2, data['LABEL'],test_size=0.25,random_state=5)

#defining the model
#compilimg the model -> we are going to use already used models GNB, MNB, CNB, BNB
#fitting the model
model1 = MNB.fit(x_train, y_train)
accuracy_score_mnb = metrics.accuracy_score(MNB.predict(x_test), y_test)
print('accuracy_score_mnb = '+str('{:4.2f}'.format(accuracy_score_mnb*100))+'%')

accuracy_score_mnb = 69.59%


In [51]:
# On Test data with training on Dataset A using TfidfVectorizer
test_text_count_2 = tfidf.transform(test_data['new'])
MNB = MultinomialNB()

# Fitting on all of the training data present
MNB.fit(text_count_2, data['LABEL'])
predicted = MNB.predict(test_text_count_2)
accuracy_score = metrics.accuracy_score(predicted, test_data['LABEL'])
print(str('{:04.2f}'.format(accuracy_score*100))+'%')

92.24%


In [52]:
import dill as pickle 
with open('a2_naivebayes_model.pkl', 'wb') as fout:
    pickle.dump(MNB, fout)

In [53]:
with open('a2_naivebayes_model.pkl', 'rb') as fin:
    ngram_loaded = pickle.load(fin)

In [ ]:
### Some examples

In [49]:
print("\n------BIGRAM RANDOMLY GENERATE SENTENCES TEST--------")
bigram_sentences = ngram.bigram_generate_sentences(50)
ngram.N = 2
for sentence in bigram_sentences:
    joined_sentence = " ".join(sentence)
    score = ngram.prob(joined_sentence)
    smoothed_score = ngram.sprob(joined_sentence)
    #print(joined_sentence, ":", score, smoothed_score)
#ppl_score = ngram.ppl(bigram_sentences)
#ppl_score_with_smoohting = ngram.ppl_unigram_smoohted(bigram_sentences)
bigram_ppl_score = ngram.ppl_bigram(bigram_sentences)
    
print("perplexity score of 5 sentences:",bigram_ppl_score) #ppl_score_with_smoohting, ppl_score,


------BIGRAM RANDOMLY GENERATE SENTENCES TEST--------
perplexity score of 5 sentences: 47.94826475553644


In [50]:
sentence = "about to bat him"
sentence = preprocess(sentence)
print("\n------BIGRAM TEST--------")
ngram.N = 2
print("The perplexity is:",ngram.ppl_bigram(sentence))


------BIGRAM TEST--------
The perplexity is: 1.0


In [62]:
'''import pandas as pd
df = pd.read_csv('C:/Users/HP/Downloads/good_turing_smooting_bigram_result.txt', sep=':',header=None)
print(df)'''

"import pandas as pd\ndf = pd.read_csv('C:/Users/HP/Downloads/good_turing_smooting_bigram_result.txt', sep=':',header=None)\nprint(df)"